In [1]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()

In [3]:
#Update replacing with put
#PUT is used to receive data that should replace the existing data.

from fastapi import FastAPI
from fastapi.encoders import jsonable_encoder
from pydantic import BaseModel

app = FastAPI()


class Item(BaseModel):
    name: str | None = None
    description: str | None = None
    price: float | None = None
    tax: float = 10.5
    tags: list[str] = []

items = {
    "foo": {"name": "Foo", "price": 50.2},
    "bar": {"name": "Bar", "description": "The bartenders", "price": 62, "tax": 20.2},
    "baz": {"name": "Baz", "description": None, "price": 50.2, "tax": 10.5, "tags": []},
}

@app.get('/items/{item_id}', response_model=Item)
async def read_item(item_id:str):
    return items[item_id]

@app.put('/items/{item_id}', response_model=Item)
async def update_item(item_id: str, item: Item):
    update_item_encoded = jsonable_encoder(item)
    items[item_id] = update_item_encoded
    return update_item_encoded

uvicorn.run(app, port=8000)






INFO:     Started server process [24182]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:60478 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:60478 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:60478 - "GET /items/foo HTTP/1.1" 200 OK
INFO:     127.0.0.1:40986 - "PUT /items/foo HTTP/1.1" 200 OK
INFO:     127.0.0.1:52748 - "GET /items/foo HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [24182]


Partial updates with PATCH

In [5]:
#Using pydantic's exclude_unset parameter

#If you want to receive partial updates, its very useful to use the parameter exclude_unset in pydantic's model's .dict()


#Like item.dict(exclude_unset = True)


from fastapi import FastAPI
from fastapi.encoders import jsonable_encoder
from pydantic import BaseModel

app = FastAPI()


class Item(BaseModel):
    name: str | None = None
    description: str | None = None
    price: float | None = None
    tax: float = 10.5
    tags: list[str] = []


items = {
    "foo": {"name": "Foo", "price": 50.2},
    "bar": {"name": "Bar", "description": "The bartenders", "price": 62, "tax": 20.2},
    "baz": {"name": "Baz", "description": None, "price": 50.2, "tax": 10.5, "tags": []},
}


@app.get("/items/{item_id}", response_model=Item)
async def read_item(item_id: str):
    return items[item_id]


@app.patch('/items/{item_id}', response_model = Item)
async def update_item(item_id: str, item: Item):
    stored_item_data = items[item_id]
    stored_item_model = Item(**stored_item_data)
    update_data = item.dict(exclude_unset=True)
    updated_item = stored_item_model.copy(update=update_data)
    items[item_id] = jsonable_encoder(updated_item)
    return updated_item


uvicorn.run(app, port=8000)

INFO:     Started server process [24182]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:47742 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:47742 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:47742 - "GET /items/bar HTTP/1.1" 200 OK
INFO:     127.0.0.1:34520 - "PATCH /items/bar HTTP/1.1" 200 OK
